1. Ingest citation data via uploaded RIS or BibTex files or via manually-entered, individual records
1. Parse citation data according to input format and standardize (across formats) field names and values as much as possible
1. Munge parsed data into a convenient format for importing into a database, e.g. CSV -> Postgres (see [here](https://www.postgresql.org/docs/9.5/static/sql-copy.html))
1. Import citations into database with additional columns for, e.g. citation_id, project_id, user_id, is_duplicate (NULL to start), confirmed_duplicate, ...
1. Apply trained dedupe model to new citations vs. existing citations for given project, find possible matches, interactively prompt user to confirm duplicates when in doubt; mark duplicate records in the db accordingly

In [6]:
import io
import logging
import os

import dedupe

import cipy

In [7]:
logger = logging.getLogger()
logging.basicConfig()

In [11]:
citations_ddl = cipy.db.get_ddl('citations')
conn_creds = cipy.db.get_conn_creds('DATABASE_URL')
citations_db = cipy.db.PostgresDB(citations_ddl, conn_creds)

In [22]:
def generate_candidates(results):

    lset = set

    block_id = None
    records = []
    i = 0
    for row in results:
        if row['block_id'] != block_id:
            if records:
                yield records

            block_id = row['block_id']
            records = []
            i += 1

            if i % 10000 == 0:
                print(i, "blocks")
                print(time.time() - start_time, "seconds")

        smaller_ids = row['smaller_ids']

        if smaller_ids:
            smaller_ids = lset(smaller_ids.split(','))
        else:
            smaller_ids = lset([])

        records.append((row['citation_id'], row, smaller_ids))

    if records:
        yield records

In [26]:
query = """
SELECT
    citation_id, authors, title, abstract,
    publication_year, doi, block_id, smaller_ids
FROM smaller_coverage 
INNER JOIN citations 
USING (citation_id)
ORDER BY block_id
"""

results = citations_db.run_query(query)

In [27]:
# for records in generate_candidates(results):
#     print(records)
#     break

In [28]:
clustered_dupes = deduper.matchBlocks(generate_candidates(results), threshold=0.5)

NameError: name 'deduper' is not defined